# 

In [ ]:
!pip install arsenic
!pip install requests_html

     |████████████████████████████████| 1.3 MB 43.0 MB/s 
     |████████████████████████████████| 49 kB 12.0 MB/s 
     |████████████████████████████████| 294 kB 70.5 MB/s 
     |████████████████████████████████| 142 kB 81.5 MB/s 
  Using cached importlib_metadata-2.1.1-py2.py3-none-any.whl (10 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.0.1
    Not uninstalling importlib-metadata at /shared-libs/python3.7/py-core/lib/python3.7/site-packages, outside environment /root/venv
    Can't uninstall 'importlib-metadata'. No files were found to uninstall.


In [ ]:
!pip install chromedriver

     |████████████████████████████████| 14.3 MB 16.4 MB/s 


In [1]:
%%writefile async_scrape.py
import os
import asyncio
from arsenic import get_session, keys, browsers, services
import pandas as pd
from requests_html import HTML 
import itertools
import re
import time
import pathlib
import pickle


async def extract_id_slug(url_path):
    regex = r"^[^\s]+/(?P<id>\d+)-(?P<slug>[\w-]+)$"
    group = re.match(regex, url_path)
    if not group:
        return None, None
    return group['id'], group['slug']

async def get_links(body_content):
    html_r = HTML(html=body_content)
    fabric_links = [x for x in list(html_r.links) if x.startswith("/en/fabric")]

    datas = []
    for path in fabric_links:
        id_, slug_ = await extract_id_slug(path)
        data = {
            'id': id_,
            'slug': slug_,
            'path': path,
            'scraped': 0
        }
        datas.append(data)
    return datas

async def scraper(url):
    service = services.Chromedriver()
    browser = browsers.Chrome()
    browser.capabilities = {
        "goog:chromeOptions": {"args": ["--headless", "--disable-gpu", "--no-sandbox", "--disable-dev-shm-usage"]}
    }
    async with get_session(service, browser) as session:
        await session.get(url)
        body = await session.get_page_source()
        # print(body)
        return body

async def store_links_as_df_pickle(datas=[], name='links.pkl'):
    df = pd.DataFrame(datas)
    df.set_index('id', drop=True, inplace=True)
    df.to_pickle(name)
    return df

async def run(url):
    body_content = await scraper(url)
    links = await get_links(body_content)
    df = await store_links_as_df_pickle(links)
    return df

if __name__ == "__main__":
    url = 'https://www.spoonflower.com/en/shop?on=fabric'
    # loop = asyncio.get_event_loop()
    # result = loop.run_until_complete(scraper(url))
    df = asyncio.run(run(url))
    print(df.head())

Overwriting async_scrape.py


In [2]:
!python async_scrape.py

Starting ChromeDriver 90.0.4430.212 (e3cd97fc771b893b7fd1879196d1215b622c2bed-refs/branch-heads/4430@{#1429}) on port 60657
Only local connections are allowed.
Please see https://chromedriver.chromium.org/security-considerations for suggestions on keeping ChromeDriver safe.
[1621507968.843][SEVERE]: bind() failed: Cannot assign requested address (99)
ChromeDriver was started successfully.
2021-05-20 10:52.48 [info     ] request                        body={"capabilities": {"alwaysMatch": {"goog:chromeOptions": {"args": ["--headless", "--disable-gpu", "--no-sandbox", "--disable-dev-shm-usage"]}}}} header={'Content-Type': 'application/json'} method=POST url=http://localhost:60657/session
2021-05-20 10:52.49 [info     ] response                       body={"capabilities": {"alwaysMatch": {"goog:chromeOptions": {"args": ["--headless", "--disable-gpu", "--no-sandbox", "--disable-dev-shm-usage"]}}}} data={'value': {'capabilities': {'acceptInsecureCerts': False, 'browserName': 'chrome', 'brow

In [3]:
import pandas as pd
name = 'links.pkl'
df = pd.read_pickle(name)
df.head()

,slug,path,scraped
id,,,
7463028,seamless-watercolor-larger-leaves-pattern-1-by...,/en/fabric/7463028-seamless-watercolor-larger-...,0
7685381,dragon-fire-by-adenaj,/en/fabric/7685381-dragon-fire-by-adenaj,0
8345549,freehand-hearts-bone-on-dusty-pink-by-erin__ke...,/en/fabric/8345549-freehand-hearts-bone-on-dus...,0
7702196,fable-floral-teal-med-by-nouveau_bohemian,/en/fabric/7702196-fable-floral-teal-med-by-no...,0
10307245,golden-girls-faces-small-teal-by-chica_and_jo,/en/fabric/10307245-golden-girls-faces-small-t...,0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=455dbd66-b743-41fb-bb10-e0d89ff02cc3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>